Since we are employing unrelaxed initial configurations for machine learning, a framework has been designed to evaluate whether adsorption configuration change after structural optimization.   

Slab structures with altered adsorption configurations or structural anomalies are filtered out to ensure the accuracy of the data.

# Adsorption configuration of a single structure
We use a function **infer_adsorption_site** to determine the adsorption configuration of a single adsorption structure

In [3]:
from heaict.data.site import infer_adsorption_site
from pymatgen.core import Structure
import os

The details of its parameters and methodology are described below.

In [4]:
infer_adsorption_site?

Signature:
infer_adsorption_site(
    slab,
    adsb=['N', 'H'],
    method='cutoff',
    judge=2.7,
    max_bond=2.7,
    defin_dist='dist',
    pymr=True,
    print_neibor=False,
)
Docstring:
Function determine the adsorption site of the molecule based on the bond length

Parameters:
    - slab (pymatgen.core.Structure): the slab structure. 
    - adsb (list): species for adsorbate. Default = ['N', 'H']
    - method (str): method for define bond judge distance. Default = 'cutoff'
      A bond whose length is less than the judge distance is considered a bond.
      'cutoff': judge distance will be the judge value itself
      'minplus': judge distance will be the minest bond distance for each adsorbate atom plus judge value
      'scalemin': judge distance will be the minest bond distance for each adsorbate atom multiply by judge
      'scaleradius': judge distance will be sum of atom radius multipy by judge. defin_dist is not work for this method.
    - judge (float): value used to h

  
You can quickly identify adsorption site and configuration by inputting an adsorption structure and specifying the element type of the adsorbing molecule.


In [6]:
slab = Structure.from_file(f'Data/Example/Site Identification/Calculated adss initial/109_H_0_0h.vasp')
site, adsb_coordinate, connection = infer_adsorption_site(slab=slab, adsb=['N', 'H'])

This is a structure with H adsorbed at a hollow site.   

After applying the function, it returns the following three values, which respectively represent:
- **site**: The atomic indices and element symbols of the adsorption site, which here consists of three atoms 21-Ni, 22-Ni, and 26-Mo.
- **adsb_coordinate**: The coordination number of each adsorbing atom to the adsorption site. In this case, there is only one H atom with a coordination of 3 to the adsorption site.
- **connection**: The atomic indices of each adsorption bond, corresponding here to three bonds, which are (28, 21), (28, 22) and (28, 26).




In [7]:
site

{'21-Ni', '22-Ni', '26-Mo'}

In [8]:
adsb_coordinate

Counter({'H-3': 1})

In [9]:
connection

[[28, 28, 28], [21, 22, 26]]

# A workflow for discovering abnormal structures
The above describes the judgment of adsorption site and configuration for individual structures.   

The evaluation and handling of adsorption configurations and structural anomalies for multiple structures have been integrated into a function **site_process**.

Before using this function, you need to prepare four folders and one data table, just as we have done in path **Data/Example/Site_Identification**.

The meanings of these folders and file are as follows:
- **Calculated_adss_error**: Folder for storing anomalous structures.
- **Calculated_adss_initial**: Initial structures of the computed slabs which corresponding to high‑representative samples selected from all constructed adsorption structures through certain sampling methods.
- **Calculated_adss_optimal**: Optimized structures of the computed slabs.
- **Constructed_adss**: All constructed adsorption slab structures.
- **data.csv**: A data table indexed by each structure's filename and contains the target quantities corresponding to each sample for subsequent machine learning..    

For details on adsorption structure construction, high‑throughput computation and data processing, initial structure sampling, and other related methodologies, please refer to my other [scripts](https://github.com/jchddd/scripts).  

## Consistency of the initial adsorption configuration

After constructing adsorption structures in batches, we aim to use a set of universal parameters for function **infer_adsorption_site** to accurately describe the adsorption sites of all initial structures.   

This is accomplished through a function **site_consistent_same_configuration**. This function evaluates whether the given parameter set for function infer_adsorption_site can correctly identify that the configurations of a molecule at a specific adsorption site type are all equivalent. If not, the parameters or initial configurations can be appropriately adjusted.   

Here, we use the default parameter set to check the configuration consistency of structures with NH adsorbed on 3-hollow sites.

In [1]:
from heaict.data.site import site_consistent_same_configuration

In [4]:
error_list = site_consistent_same_configuration(
    file_path=f'Data/Example/Site_Identification/Calculated_adss_initial/',
    adsorbate='NH',
    adsorption_site='h',
    sample='109_NH_0_0h.vasp',
    disable_tqdm=False,
    jupyter_tqdm=True,
    print_error_info=True,
    #method='cutoff', ... other parameters from infer_adsorption_site
)

Check the site determination of slabs with same adsorption configuration
Expected result:
    site:  hollow {'22-Ni', '26-Mo', '21-Ni'}
    coor:  Counter({'H-0': 1, 'N-3': 1})


  0%|          | 0/18 [00:00<?, ?it/s]

error/total:  0 / 3


## Batch detection through function site_process
After confirming that the adsorption configuration of  initial structures can be accurately identified and preparing the above-mentioned folders and file, the configuration can be detected in batches through the function **site_process**.   

First, let's take a look at what parameters this function contains.

In [12]:
from heaict.data.site import site_process
from shutil import copy, move
import pandas as pd

In [11]:
site_process?

Signature:
site_process(
    path_all_ori,
    path_ori,
    path_opt,
    para_infer_site_origin={},
    para_infer_site_optima=[{}],
    isomorphism_ori=[],
    isomorphism_opt=[],
    cutoff_multiplier=1.3,
    adsb_symbol=['N', 'H'],
    uniform_adsb={'N2h': 'N2', 'N2v': 'N2', 'NNHh': 'NNH', 'NNHv': 'NNH', 'NH': 'NH', 'NH3': 'NH3', 'H': 'H'},
    deal_file=False,
    error_path=None,
    dataframe=None,
    detect_anomaly=True,
    disable_tqdm=True,
    jupyter_tqdm=True,
)
Docstring:
A process that combines anomaly detection, site change detection, and finding initial configurations.

Parameters:
    - path_all_ori (path): path to all constructed initial slab structures.
    - path_ori (path): path to calculated initial slab structures.
    - path_opt (path): path to relaxed slab structures.
    - para_infer_site_origin (dict): a dict of key value pairs of parameter on infer_adsorption_site for origin slab. Default = {}
    - para_infer_site_optima (list of dict): a list of dicts

Before continuing to run the notebook, restore the processed data to its original state.

In [34]:
move(f'Data/Example/Site_Identification/Calculated_adss_error/109_N2v_0_6t_initial.vasp', f'Data/Example/Site_Identification/Calculated_adss_initial/109_N2v_0_6t.vasp')
move(f'Data/Example/Site_Identification/Calculated_adss_error/109_N2v_0_6t.vasp', f'Data/Example/Site_Identification/Calculated_adss_optimal/109_N2v_0_6t.vasp')
move(f'Data/Example/Site_Identification/Calculated_adss_error/109_NNHh_270_4h_initial.vasp', f'Data/Example/Site_Identification/Calculated_adss_initial/109_NNHh_270_4h.vasp')
move(f'Data/Example/Site_Identification/Calculated_adss_error/109_NNHh_270_4h.vasp', f'Data/Example/Site_Identification/Calculated_adss_optimal/109_NNHh_270_4h.vasp')
os.remove(f'Data/Example/Site_Identification/Calculated_adss_initial/109_NNHh_90_0h.vasp')
copy(f'Data/Example/Site_Identification/Constructed_adss_all/109_NNHh_60_1b.vasp', f'Data/Example/Site_Identification/Calculated_adss_initial/109_NNHh_60_1b.vasp')
move(f'Data/Example/Site_Identification/Calculated_adss_optimal/109_NNHh_90_0h.vasp', f'Data/Example/Site_Identification/Calculated_adss_optimal/109_NNHh_60_1b.vasp')

'Data/Example/Site_Identification/Calculated_adss_optimal/109_NNHh_60_1b.vasp'

Now let's try running this method. This methd works as follows:
- The method will first detect structural anomalies such as molecular decomposition and surface reconstruction. This part is done by related functions from [fairchem](https://fair-chem.github.io/index.html).
- Afterwards, it will use function **infer_adsorption_site** to determine whether the adsorption configuration of the structure has changed before and after optimization.
- If the adsorption configuration has changed, it will attempt to find a structure among all constructed structures that matches the adsorption configuration of the optimized structure. If no matching structure is found, it will be classified as an erroneous structure.

Among the 20 structures included in the case study, there is one anomalous structure with molecular decomposition, and two structures where the adsorption site has changed. Among these two, one can be matched with a corresponding initial structure.
- 109_NNHh_270_4h: molecular decomposition
- 109_NNHh_60_1b: adsorption configuration changes with a matched initial structure
- 109_N2v_0_6t: adsorption configuration changes without a matched initial structure

In [35]:
path_error = f'Data/Example/Site_Identification/Calculated_adss_error/'
path_initial = f'Data/Example/Site_Identification/Calculated_adss_initial/'
path_optimal = f'Data/Example/Site_Identification/Calculated_adss_optimal/'
path_all = f'Data/Example/Site_Identification/Constructed_adss_all/'
df = pd.read_csv(f'Data/Example/Site_Identification/data.csv', index_col=0)

In [36]:
df_anomaly, error_list, isomo_list, df_suggest, empty_list = site_process(
    path_all_ori=path_all,
    path_ori=path_initial,
    path_opt=path_optimal,
    para_infer_site_origin={},
    para_infer_site_optima=[
        {'judge': 0.6, 'method': 'cutoff', 'defin_dist': 'radius', 'max_bond': 3},
        {'judge': 0.6, 'method': 'minplus', 'defin_dist': 'radius', 'max_bond': 3},
        {'judge': 0.6, 'method': 'cutoff', 'defin_dist': 'radius', 'pymr': False, 'max_bond': 3},
        {'judge': 0.6, 'method': 'minplus', 'defin_dist': 'radius', 'pymr': False, 'max_bond': 3},
        {'judge': 2.7},
        {'judge': 1.5, 'max_bond': 3, 'method': 'scaleradius'},
        {'judge': 1.5, 'max_bond': 3, 'method': 'scaleradius', 'pymr': False},   
    ],
    adsb_symbol=['N', 'H'],
    uniform_adsb={'N2h': 'N2', 'N2v': 'N2', 'NNHh': 'NNH', 'NNHv': 'NNH', 'NH': 'NH', 'NH3': 'NH3', 'H': 'H'},
    deal_file=True,
    error_path=path_error,
    dataframe=df,
    detect_anomaly=True,
    disable_tqdm=False,
    jupyter_tqdm=True
)

Start detect anomaly


  0%|          | 0/20 [00:00<?, ?it/s]

Find 1 anomaly slabs.
Move anomaly slabs to error path
Check site consistent for same slab


  0%|          | 0/19 [00:00<?, ?it/s]

Consider 0 slabs as isomorphism
Find 2 slabs with change site
0 slabs have not initial slab
Suggest initial slab for site change slab


  0%|          | 0/2 [00:00<?, ?it/s]

Find 1 suggest slab
0 recommendation slabs already exist
Move site change slabs to error path
Replace 1 slabs with recommendation slab
Complete the deletion and correction of the dataframe


In [37]:
df_anomaly

,adsorbate_dissociated,adsorbate_desorbed,surface_changed,adsorbate_intercalated
109_NNHh_270_4h.vasp,True,False,False,False


In [38]:
error_list

['109_N2v_0_6t.vasp']

In [39]:
df_suggest

,initial adss,suggest adss,already exist
0,109_NNHh_60_1b.vasp,109_NNHh_90_0h.vasp,False


Finally, save the modified data table. If the data table is passed in, all modifications will be synchronized to it. Here, we have removed 2 anomalous structures, reducing the data table from the original 20 entries to 18.



In [20]:
df.shape

(18, 3)

In [21]:
df.to_csv(f'Data/Example/Site_Identification/data2.csv')

Other points to note include:
- Due to significant structural change after optimization, you can input multiple sets of parameters for function infer_adsorption_site to determine the adsorption configuration on parameter **para_infer_site_optima**. As long as one set of parameters can identify an adsorption configuration consistent with the initial configuration, it will be considered that the configuration has not changed.
- If multiple sets of parameters are still insufficient, parameters **isomorphism_ori** and **isomorphism_opt** can be used. The combined use of these two parameters allows structures with the same adsorption site (same adsorption site indices and elements) but different adsorbate coordination numbers (**adsb_coordinate**) to be considered as the same configuration type.
- Parameter **uniform_adsb** is used to specify the search scope for the initial configurations of structures that have undergone configuration changes. Only structures with consistent molecular types will be searched. **This also eliminates additional requirements, such as ensuring that the structure names you use follow the same naming conventions as in my workflow. A slab structure is named by '(slab system name)\_(adsorbate name)\_(rotation of the adsorbate)\_(index and type of the adsorption site)', such slab strucutres can be directly generated from this [script](https://github.com/jchddd/scripts/blob/main/jworkflow/Tutorial-slab.ipynb).**
- Detecting anomalous structures requires a significant amount of time. You can disable it using parameter **detect_anomaly**. When disabled, output variable **df_anomaly** should also be removed.